<a href="https://colab.research.google.com/github/ergul13/RAG/blob/main/RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install langchain openai faiss-cpu python-dotenv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 55.3 MB/s eta 0:00:00


In [2]:
import os
from google.colab import userdata

# Colab'in 'secrets' bölümünden API anahtarını alıyoruz.
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')

# LangChain gibi kütüphanelerin bu anahtarı otomatik olarak bulabilmesi için
# onu bir "çevre değişkeni" olarak ayarlıyoruz.
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY

print("API Anahtarı başarıyla yüklendi ve ayarlandı!")

API Anahtarı başarıyla yüklendi ve ayarlandı!
